In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
import pandas as pd
from os import walk
import tensorflow as tf
import tensorflow_probability as tfp
from sklearn import preprocessing
# from keras import activations, initializers
# from keras.layers import Layer
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import pickle
import h5py
from tensorflow.keras.models import load_model

In [ ]:
include_wave = 'no'
modelID = 3
# (0 for SCADA only, 1 for SCADA+Acc17, 2 for SCADA+Acc38, 3 for SCADA+Acc77, 
# 4 for SCADA+Acc17&38, 5 for SCADA+Acc17&38&77 
duration = '24M'

def NLL(y, distr): 
  return -distr.log_prob(y) 

def normal_sp(params): 
  return tfp.distributions.Normal(loc=params[:,0:2], scale=1e-3 
                                  + tf.math.softplus(0.05 * params[:,2:4]))# both parameters are learnable

kernel_divergence_fn=lambda q, p, _: tfp.distributions.kl_divergence(q, p) / (52803 * 1.0)
bias_divergence_fn=lambda q, p, _: tfp.distributions.kl_divergence(q, p) / (52803 * 1.0)

inputs = tf.keras.layers.Input(shape=(13,))

hidden = tfp.layers.DenseFlipout(32,bias_posterior_fn=tfp.layers.util.default_mean_field_normal_fn(),
                           bias_prior_fn=tfp.layers.default_multivariate_normal_fn,
                           kernel_divergence_fn=kernel_divergence_fn,
                           bias_divergence_fn=bias_divergence_fn,activation="relu")(inputs)
hidden = tfp.layers.DenseFlipout(64,bias_posterior_fn=tfp.layers.util.default_mean_field_normal_fn(),
                           bias_prior_fn=tfp.layers.default_multivariate_normal_fn,
                           kernel_divergence_fn=kernel_divergence_fn,
                           bias_divergence_fn=bias_divergence_fn,activation="relu")(hidden)
hidden = tfp.layers.DenseFlipout(32,bias_posterior_fn=tfp.layers.util.default_mean_field_normal_fn(),
                           bias_prior_fn=tfp.layers.default_multivariate_normal_fn,
                           kernel_divergence_fn=kernel_divergence_fn,
                           bias_divergence_fn=bias_divergence_fn,activation="relu")(hidden)
params = tfp.layers.DenseFlipout(4,bias_posterior_fn=tfp.layers.util.default_mean_field_normal_fn(),
                           bias_prior_fn=tfp.layers.default_multivariate_normal_fn,
                           kernel_divergence_fn=kernel_divergence_fn,
                           bias_divergence_fn=bias_divergence_fn)(hidden)
dist = tfp.layers.DistributionLambda(normal_sp)(params)


model = Model(inputs=inputs, outputs=dist)
model.compile(Adam(learning_rate=0.0002), loss=NLL) 

model_params = Model(inputs=inputs, outputs=params)
# model.summary()

file = h5py.File('Weights/02_PredictionModel/Model%d_IncWave%s_%s.h5' % (modelID, include_wave, duration), 'r')
weight = []
for i in range(len(file.keys())):
   weight.append(file['weight' + str(i)][:])
model.set_weights(weight)

In [ ]:
def compute_predictions_pbnn(model, samples):
    prediction_distribution= model(samples)
    prediction_mean = np.squeeze(prediction_distribution.mean().numpy())/10
    prediction_stdv = np.squeeze(prediction_distribution.stddev().numpy())/10

    # The 95% CI is computed as mean ± (1.96 * stdv)
    upper = (prediction_mean + (1.96 * prediction_stdv))
    lower = (prediction_mean - (1.96 * prediction_stdv))

    
    return prediction_mean, prediction_stdv, upper, lower

def loglikelihood(y, loc, scale):
    dist = tfp.distributions.Normal(loc, scale)
    return -dist.log_prob(y)

In [ ]:
# Normlaization of input data
# Data normalization according to training dataset/ model
filehandler = open('Weights/Norm', 'rb') 
std_scaler = pickle.load(filehandler)

In [ ]:
# mp02
test_input = pd.read_pickle('DATA/mp02df_input')
index = test_input.columns
test_output = pd.read_pickle('DATA/mp02df_output')

#In training samples
X1 = np.zeros([5, 13]) 
Y1 = np.zeros([5, 2])
intrain_input = test_input.loc[((test_input['windspeed'] == 10) | (test_input['windspeed'] == 12.6) 
                            | (test_input['windspeed'] == 15.5) | (test_input['windspeed'] == 16.9) | 
                            (test_input['windspeed'] == 20))
                            & (test_input['max_TW_ACC_LAT077_FA'] > 0.02)
                            & (test_input['max_TW_ACC_LAT077_FA'] <= 0.03)]
intrain_input = intrain_input.sort_values(by=['windspeed'])
intrain_input = intrain_input.drop_duplicates(subset=['windspeed'], keep='last')
print(intrain_input.shape)
intrain_output = test_output.reindex(intrain_input.index)
print(intrain_output.shape)
# Retrive features based on the modelID_
index1 = pd.core.indexes.base.Index([]) # create a blank index array
if modelID == 3: # Acc77
    index1 = index1.append(index[[7,8,13,14,19,20]])
index1 = index1.append(index[21:]) # SCADA

# Normalization
inputn = pd.DataFrame(std_scaler.transform(intrain_input), columns=intrain_input.columns) 
outputn = intrain_output/10**6  #
X1= inputn[index1].values
Y1= outputn.values

#Out of training samples
X2 = np.zeros([5, 13]) 
Y2 = np.zeros([5, 2])
outtrain_input = test_input.loc[((test_input['windspeed'] == 10) | (test_input['G08_windspeed'] == 12.6) 
                            | (test_input['windspeed'] == 15.5) | (test_input['windspeed'] == 16.9) | 
                            (test_input['windspeed'] == 20))
                            & (test_input['max_TW_ACC_LAT077_FA'] > 0.15)
                            & (test_input['max_TW_ACC_LAT077_FA'] <= 0.3)]
outtrain_input = outtrain_input.sort_values(by=['windspeed'])
outtrain_input = outtrain_input.drop_duplicates(subset=['windspeed'], keep='last')
print(outtrain_input)
outtrain_output = test_output.reindex(outtrain_input.index)
print(outtrain_output.shape)
# Normalization
inputn = pd.DataFrame(std_scaler.transform(outtrain_input), columns=outtrain_input.columns) 
outputn = outtrain_output/10**6  #
X2= inputn[index1].values
Y2= outputn.values

In [ ]:
print(X1)
print(Y1)
print(X2)
print(Y2)

In [ ]:
# Epistemic uncertainty
   
# cm = 1/2.54   
# plt.rcParams["font.family"] = "Times New Roman"
# plt.rcParams["font.size"] = 9

# cv_deml = np.zeros([100, 3])
# cv_demn = np.zeros([100, 3])
# for j in range(100):
#     prediction_mean, prediction_stdv, upper, lower = compute_predictions_pbnn(model, X1)
#     cv_deml[j, :] = prediction_stdv[:,0]/prediction_mean[:,0]
#     cv_demn[j, :] = prediction_stdv[:,1]/prediction_mean[:,1]
# M1_stdl = np.mean(cv_deml, axis = 0)
# M1_stdn = np.mean(cv_demn, axis = 0)
# V1_stdl = np.std(cv_deml, axis = 0)
# V1_stdn = np.std(cv_demn, axis = 0)
# print('MU = ',  V1_stdl/M1_stdl)
# print('MU = ',  V1_stdn/M1_stdn)

# cv_deml = np.zeros([100, 3])
# cv_demn = np.zeros([100, 3])
# for j in range(100):
#     prediction_mean, prediction_stdv, upper, lower = compute_predictions_pbnn(model, X2)
#     cv_deml[j, :] = prediction_stdv[:,0]/prediction_mean[:,0]
#     cv_demn[j, :] = prediction_stdv[:,1]/prediction_mean[:,1]
# M2_stdl = np.mean(cv_deml, axis = 0)
# M2_stdn = np.mean(cv_demn, axis = 0)
# V2_stdl = np.std(cv_deml, axis = 0)
# V2_stdn = np.std(cv_demn, axis = 0)
# print('MU = ',  V2_stdl/M2_stdl)
# print('MU = ',  V2_stdn/M2_stdn)



In [ ]:
# Predictive uncertainty
   
cm = 1/2.54   
plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams["font.size"] = 9

deml = np.zeros([100, len(Y1)])
demn = np.zeros([100, len(Y1)])
for j in range(100):
    dems = model.predict(X1)
    deml[j, :] = dems[:,0]/10
    demn[j, :] = dems[:,1]/10
M1_l = np.mean(deml, axis = 0)
M1_n = np.mean(demn, axis = 0)
V1_l = np.std(deml, axis = 0)
V1_n = np.std(demn, axis = 0)
print('PU = ',  V1_stdl/M1_stdl)
print('PU = ',  V1_stdn/M1_stdn)

deml = np.zeros([100, len(Y1)])
demn = np.zeros([100, len(Y1)])
for j in range(100):
    dems = model.predict(X2)
    deml[j, :] = dems[:,0]/10
    demn[j, :] = dems[:,1]/10
M2_l = np.mean(deml, axis = 0)
M2_n = np.mean(demn, axis = 0)
V2_l = np.std(deml, axis = 0)
V2_n = np.std(demn, axis = 0)
print('PU = ',  V2_stdl/M2_stdl)
print('PU = ',  V2_stdn/M2_stdn)

In [ ]:
# Deterministic Model
VSnet = load_model('Weights/01_SensorPlacementTest/DNNModel%s_IncWave%s.h5' % (modelID, include_wave))
dnn_output1 = VSnet.predict(X1)
dnn_output2 = VSnet.predict(X2)

In [ ]:
cm = 1/2.54  # centimeters in inches
plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams["font.size"] = 9
x_test = np.array([10, 12.6, 14.9, 16.9, 20])
fig1, ax = plt.subplots(2, figsize=(8.5*cm, 7*cm), sharey='row', dpi=80, facecolor='w', edgecolor='k')
plt.subplots_adjust(left=0.15, right=.98, top=0.98, bottom=0.15, hspace = 0.1, wspace=0.15)
ax[0].scatter(x_test,Y1[:,0], color = '#cb181d', marker='o', facecolor='white')
ax[0].scatter(x_test,dnn_output1[:,0], marker = 'x', color = 'black')
ax[0].errorbar(x_test,M1_l, 1.96*V1_l, marker='_', linestyle='none', capsize=5, label=r'Prediction (Mean $\pm$ 1.96 Std)')
ax[1].scatter(x_test,Y1[:,1], color = '#cb181d', marker='o', facecolor='white')
ax[1].scatter(x_test,dnn_output1[:,1], marker = 'x', color = 'black')
ax[1].errorbar(x_test,M1_n, 1.96*V1_n, marker='_', linestyle='none', capsize=5, label=r'Prediction (Mean $\pm$ 1.96 Std)')
ax[0].set_yticklabels([])
ax[0].set_xticklabels([])
ax[0].set_ylim([0,0.65])
ax[0].set_ylabel('DEM$_{tl}$')   
ax[1].set_yticklabels([])
ax[1].set_ylim([0,0.65])
ax[1].set_ylabel('DEM$_{tn}$')
ax[1].set_xlabel('Wind speed (m/s)')
ax[0].legend(['Measured', 'DNN', 'BNN'], ncol=3, loc = 'upper left')
fig1.savefig('Figures/04_FarmwideApplication/MP02Intrain_DNN_BNN.pdf')

fig2, ax = plt.subplots(2, figsize=(8.5*cm, 7*cm), sharey='row', dpi=80, facecolor='w', edgecolor='k')
plt.subplots_adjust(left=0.15, right=.98, top=0.98, bottom=0.15, hspace = 0.1, wspace=0.15)
ax[0].scatter(x_test, Y2[:,0], color = '#cb181d', marker='o', facecolor='white')
ax[0].scatter(x_test, dnn_output2[:,0], marker = 'x', color = 'black')
ax[0].errorbar(x_test, M2_l, 1.96*V2_l, marker='_', linestyle='none', capsize=5, label=r'Prediction (Mean $\pm$ 1.96 Std)')
ax[1].scatter(x_test,Y2[:,1], color = '#cb181d', marker='o', facecolor='white')
ax[1].scatter(x_test,dnn_output2[:,1], marker = 'x', color = 'black')
ax[1].errorbar(x_test,M2_n, 1.96*V2_n, marker='_', linestyle='none', capsize=5, label=r'Prediction (Mean $\pm$ 1.96 Std)')
ax[0].set_yticklabels([])
ax[0].set_xticklabels([])
ax[0].set_ylim([0,0.65])
ax[0].set_ylabel('DEM$_{tl}$')   
ax[1].set_yticklabels([])
ax[1].set_ylim([0,0.65])
ax[1].set_ylabel('DEM$_{tn}$')
ax[1].set_xlabel('Wind speed (m/s)')
ax[0].legend(['Measured', 'DNN', 'BNN'], ncol=3, loc = 'upper left')
fig2.savefig('Figures/04_FarmwideApplication/MP02Outtrain_DNN_BNN.pdf')